In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os, sys, ssl
# Elimina le variabili di ambiente proxy
proxy_vars = ['http_proxy', 'https_proxy', 'HTTP_PROXY', 'HTTPS_PROXY', 'all_proxy', 'ALL_PROXY']

for var in proxy_vars:
   os.environ.pop(var, None) 

ssl._create_default_https_context = ssl._create_unverified_context
# Add the project root directory to the system path
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

from trading_system_py.retriever.fetch_data import FetchSingleStock, DataVisualizationStock

## INTRODUZIONE AI PREZZI E ALLA PREVEDIBILITÀ

### Il modello decompositivo classico
- I prezzi di un asset finanziario possono essere rappresentati da:  
  `P_t = segnale + errore`
  - **Segnale**: Parte prevedibile, legata a trend o cicli strutturati.  
  - **Errore**: Parte casuale, legata a fattori imprevedibili come notizie o volatilità.  

### Errore casuale e teoria del Random Walk
- Secondo la teoria del Random Walk (RW), i prezzi si muovono in modo casuale a causa dell'errore.  
- Questo rende i rendimenti finanziari difficilmente prevedibili.  

### Componenti deterministiche prevedibili
- Esistono componenti prevedibili come:
  - Trend persistenti.  
  - Pattern ripetuti.  
  - Fenomeni riconoscibili attraverso strumenti avanzati o informazioni superiori.  

## IL TREND COME COMPONENTE PREVEDIBILE

### Cos’è il trend?
- Il trend è la direzione generale del movimento dei prezzi su un determinato periodo:
  - **Rialzista**: Prezzi in aumento.  
  - **Ribassista**: Prezzi in calo.  
  - **Laterale**: Prezzi stabili.  

### Metodi per individuare il trend
1. **Modelli e pattern grafici**: Identificazione visiva di supporti, resistenze e livelli chiave.  
2. **Modelli analitici**: Utilizzo di strumenti matematici e statistici.  
3. **Esperienza del trader**: Analisi del sentiment e correlazioni tra mercati (commodities, opzioni, azionario).  

### Mercati bull vs bear
- **Mercati bull**: Trend rialzisti più lunghi e prevedibili.  
- **Mercati bear**: Trend ribassisti più brevi e difficili da seguire.  

In [ ]:
# Dizionario per le stock
stock_dict = {
    "AAPL": "Apple Inc. - Leader nel settore tecnologico, offre solidi fondamentali e una storia di crescita costante.",
    "MSFT": "Microsoft Corporation - Azienda diversificata con forte presenza nel cloud computing e software.",
    "AMZN": "Amazon.com Inc. - Gigante dell'e-commerce e dei servizi cloud, con potenziale di crescita a lungo termine.",
    "GOOGL": "Alphabet Inc. (Classe A) - Holding di Google, dominante nel settore della pubblicità online e dei servizi digitali.",
    "TSLA": "Tesla Inc. - Pioniere nel settore dei veicoli elettrici, con forte innovazione tecnologica.",
    "JNJ": "Johnson & Johnson - Azienda leader nel settore farmaceutico e dei dispositivi medici, con una solida storia di dividendi.",
    "PG": "Procter & Gamble Co. - Multinazionale con un ampio portafoglio di marchi noti, stabile e resiliente.",
    "KO": "Coca-Cola Company - Marchio globale con una forte presenza nel mercato delle bevande analcoliche.",
    "XOM": "Exxon Mobil Corporation - Una delle maggiori compagnie petrolifere integrate a livello mondiale.",
    "JPM": "JPMorgan Chase & Co. - Banca leader con una vasta gamma di servizi finanziari e una solida posizione di mercato.",
    "WMT": "Walmart Inc. - Il più grande rivenditore al mondo, con una vasta rete di negozi e una crescente presenza online.",
    "V": "Visa Inc. - Leader nei pagamenti elettronici, beneficiando della transizione verso una società senza contanti.",
    "MCD": "McDonald's Corporation - Catena di fast food globale con un modello di business consolidato e una forte presenza internazionale.",
    "INTC": "Intel Corporation - Uno dei principali produttori di semiconduttori, con una lunga storia nel settore tecnologico."
}

# Dizionario per i bond
bond_dict = {
    "LQD": "iShares iBoxx $ Investment Grade Corporate Bond ETF - Offre esposizione a obbligazioni corporate di alta qualità creditizia.",
    "TLT": "iShares 20+ Year Treasury Bond ETF - Fornisce accesso a titoli di Stato USA a lunga scadenza, utile per diversificazione e gestione del rischio.",
    "BND": "Vanguard Total Bond Market ETF - Rappresenta l'intero mercato obbligazionario statunitense, ideale per una copertura ampia.",
    "EMB": "iShares J.P. Morgan USD Emerging Markets Bond ETF - Offre esposizione a obbligazioni dei mercati emergenti, aggiungendo potenziale rendimento e diversificazione.",
    "HYG": "iShares iBoxx $ High Yield Corporate Bond ETF - Fornisce accesso a obbligazioni corporate ad alto rendimento, sebbene con maggiore rischio."
}

# Dizionario per gli ETF
etf_dict = {
    "SPY": "SPDR S&P 500 ETF Trust - Replica l'indice S&P 500, offrendo esposizione alle principali società statunitensi.",
    "EEM": "iShares MSCI Emerging Markets ETF - Offre accesso ai mercati azionari emergenti, utile per diversificazione geografica.",
    "VEA": "Vanguard FTSE Developed Markets ETF - Fornisce esposizione ai mercati sviluppati al di fuori degli Stati Uniti.",
    "QQQ": "Invesco QQQ Trust - Replica l'indice Nasdaq-100, concentrato su aziende tecnologiche di grande capitalizzazione.",
    "IWM": "iShares Russell 2000 ETF - Offre esposizione alle piccole capitalizzazioni statunitensi, aggiungendo diversificazione al portafoglio.",
    "XLF": "Financial Select Sector SPDR Fund - Fornisce esposizione al settore finanziario statunitense, utile per strategie settoriali.",
    "XLK": "Technology Select Sector SPDR Fund - Concentra l'investimento nel settore tecnologico, ideale per capitalizzare sull'innovazione.",
    "XLE": "Energy Select Sector SPDR Fund - Offre accesso al settore energetico, utile in fasi di rialzo dei prezzi delle materie prime.",
    "XLV": "Health Care Select Sector SPDR Fund - Fornisce esposizione al settore sanitario, considerato difensivo in periodi di volatilità.",
    "XLY": "Consumer Discretionary Select Sector SPDR Fund - Offre accesso al settore dei beni di consumo discrezionali, sensibile ai cicli economici."
}

## Utilizzo delle classi di retrieving del dato

Questa libreria Python è progettata per facilitare l’estrazione e la gestione di dati finanziari relativi a singoli titoli e portafogli. Utilizza l’**API Yahoo Finance** (yfinance) per scaricare serie storiche di prezzi e dati fondamentali, oltre a fornire funzionalità per la gestione di dati risk-free e la manipolazione di dati finanziari. 

La libreria include anche funzionalità per calcolare se un titolo è sottovalutato o sopravvalutato basandosi sul rapporto prezzo/utili (P/E).

In [ ]:
retriever = FetchSingleStock()
stock = retriever.download_history(ticker='SPY', date_range=['2000-01-01', '2024-10-01'])
stock.head(10)

In [ ]:
retriever.download_options_put_call_ratio(ticker='AAPL')

In [ ]:
retriever.resample_history_by_custom_bar_time_interval(stock, time_interval='ME').head(10)

## MODELLI E PATTERN GRAFICI

### Grafici finanziari
- Rappresentano i prezzi nel tempo:  
  - **Ascissa**: Tempo.  
  - **Ordinata**: Prezzo.  

#### Caratteristiche dei grafici
- **Unità di tempo**: Intraday, giornaliere, settimanali, mensili.  
- **Scala**: Lineare o logaritmica.  
- **Tipologie**:  
  - Lineare: Connette i prezzi di chiusura.  
  - A barre: Mostra apertura, chiusura, massimo e minimo.  
  - A candele: Evidenzia fasi rialziste/ribassiste.

### Livelli di prezzo
- **Statici**:  
  - **Supporti**: Livelli in cui il prezzo tende a rimbalzare.  
  - **Resistenze**: Livelli in cui il prezzo tende a invertire verso il basso.  
- **Dinamici**:  
  - **Trendline**: Linee che seguono massimi/minimi crescenti o decrescenti.  
  - **Returnline**: Parallela alla trendline, forma un canale di prezzo.  
- **Break-out**: Rottura di supporti/resistenze.  
- **Gap**: Salti improvvisi del prezzo senza scambi intermedi.

### Analisi tecnica
- Principio: Tutte le informazioni rilevanti sono riflesse nel prezzo.  
- **Pattern comuni**:  
  - Rettangoli, Testa e Spalle, Triangoli, Bandiere, Cunei.  


---

## MODELLI ANALITICI

### Analisi di trend e volatilità
- I modelli analitici stimano il miglior **processo generatore dei dati** (DGP).  
- Formula base:  
  `Y = Segnale + Errore`

### Strumenti principali
1. **Medie mobili**: Filtrano il rumore per evidenziare il trend.  
2. **Oscillatori**:  
   - **Momentum**: Velocità del movimento.  
   - **ROC**: Variazione percentuale.  
   - **MACD**: Trend e momentum.  
   - **RSI e Stocastico**: Ipercomprato/ipervenduto.  
3. **Tecniche statistiche**:  
   - Smoothing esponenziale, Bande di Bollinger, Analisi delle serie storiche.  

In [ ]:
visualizer = DataVisualizationStock(stock)
visualizer.plot_ohlc()

In [ ]:
from trading_system_py.utils.technical_indicators import TechnicalIndicators

talib = TechnicalIndicators()

for x in talib.get_all_available_indicators():
    print(x, talib.get_function_signature(func_name=x))

In [ ]:
f = 'SMA'
talib.get_function_signature(func_name=f).parameters.keys()

In [ ]:
from trading_system_py.utils.technical_indicators import TechnicalIndicatorsDataObject
technical_indicators = TechnicalIndicatorsDataObject(
            open_prices=stock['Open'], 
            high_prices=stock['High'], 
            low_prices=stock['Low'], 
            close_prices=stock['Close'], 
            volumes=stock['Volume']
        )

In [ ]:
technical_indicators.calculate_all_indicators(as_pandas=True)

In [ ]:
tmp = technical_indicators.calculate_all_indicators(specific_indicator_list=[('SMA_5', 'SMA', {'timeperiod': 5}),
                                                                             ('SMA_10', 'SMA', {'timeperiod': 10})], as_pandas=True)

tmp.head(10)

In [ ]:
visualizer.plot_combined([
    ('OHLC', {}),
    #('SMA', {'timeperiod': 200}),
    #('T3', {}),
    ('VOLUME', {}),
    ('MACD', {}),
    #('RSI', {'period': 20}),
    ('BBANDS', {})
    ])

In [ ]:
risk_free_rate = retriever.download_risk_free_rate_curve(ticker='^IRX', date_range=['2014-01-01', '2024-10-01'])
risk_free_rate.head()

In [ ]:
retriever.scrape_fundamental_data(ticker='AAPL')

In [ ]:
retriever.download_options_put_call_ratio(ticker='AAPL')

In [ ]:
from trading_system_py.retriever.fetch_data import FetchData


data_retriever = FetchData(ticker_list = list(stock_dict.keys())+list(bond_dict.keys()), 
                           date_range=['2014-01-01', '2024-10-01'])

In [ ]:
data_retriever.data

In [ ]:
data_retriever.get_all_fundamental()

In [ ]:
data_retriever.get_all_put_call_ratio(as_pandas=False)

In [ ]:
data_retriever.get_all_risk_free_curve()

In [ ]:
from trading_system_py.retriever.import_data_from_object import ImportDataFromObject

data_path = os.path.abspath(os.path.join(os.getcwd(), '../data'))
data_retriever.export(data_path, filename='sample_30_min.pickle')


data_from_pickle = ImportDataFromObject(path=data_path, filename='sample_30_min.pickle')
data_from_pickle.__dict__

In [ ]:
import matplotlib.pyplot as plt

visualizer = DataVisualizationStock(data_retriever.get_all_risk_free_curve())
visualizer.plot_ohlc()
plt.plot(data_retriever.get_all_risk_free_curve()['daily'])

In [ ]:
data_retriever.plot_ticker_data(ticker='AMZN', plots=[
    ('OHLC', {}),
    ('SMA', {'timeperiod': 50}),
    ('SMA', {'timeperiod': 200}),
    #('T3', {}),
    ('VOLUME', {}),
    #('MACD', {}),
    #('RSI', {'period': 20}),
    ('BBANDS', {})
    ])

## PORTAFOGLIO E FRONTIERA EFFICIENTE

### Cos’è un portafoglio?
- Combinazione di più asset definito secondo la propria propensione al rischio.  
- **Operazioni principali**: Acquisto, vendita, vendita allo scoperto, cover short.  

### Frontiera efficiente
- Rappresenta il miglior mix rischio-rendimento.  
- **Misura chiave**: Sharpe Ratio (rendimento corretto per il rischio). 

## **Frontiera Efficiente: Estensione Teorica**

### **Cos’è la Frontiera Efficiente?**
La frontiera efficiente è un concetto fondamentale della teoria del portafoglio moderno (Modern Portfolio Theory, MPT) sviluppata da Harry Markowitz. Essa rappresenta l'insieme di tutti i portafogli ottimali che offrono il massimo rendimento atteso per un dato livello di rischio o, viceversa, il minimo rischio per un dato livello di rendimento atteso.

### **Caratteristiche Principali**
1. **Rischio e Rendimento:**  
   Ogni portafoglio può essere valutato in base a due parametri:
   - **Rendimento atteso**: La media dei ritorni futuri previsti.
   - **Rischio**: Spesso misurato come deviazione standard (volatilità) dei rendimenti.

2. **Efficienza:**  
   Un portafoglio è considerato efficiente se:
   - Non esiste un altro portafoglio con lo stesso livello di rischio e un rendimento maggiore.
   - Non esiste un altro portafoglio con lo stesso livello di rendimento e un rischio minore.

3. **Dominanza:**  
   - Portafogli al di sotto della frontiera efficiente sono inefficaci, in quanto esistono alternative migliori in termini di rischio e rendimento.
   - La frontiera rappresenta quindi il limite superiore degli investimenti ottimali.

---

### **Costruzione della Frontiera Efficiente**
1. **Calcolo del Rendimento Atteso del Portafoglio (\(E(R_p)\)):**
   \[
   $$E(R_p) = \sum_{i=1}^n w_i E(R_i)$$
   \]
   Dove:
   - $E(R_p)$: Rendimento atteso del portafoglio.
   - $w_i$: Peso dell’asset \(i\) nel portafoglio.
   - $E(R_i)$: Rendimento atteso dell’asset \(i\).

2. **Calcolo del Rischio del Portafoglio (\(\sigma_p\)):**
   \[
   $$\sigma_p^2 = \sum_{i=1}^n \sum_{j=1}^n w_i w_j \text{Cov}(R_i, R_j)$$
   \]
   Dove:
   - $\sigma_p^2$: Varianza del portafoglio.
   - $w_i, w_j$: Pesi degli asset \(i\) e \(j\).
   - $\text{Cov}(R_i, R_j)$: Covarianza tra i rendimenti degli asset \(i\) e \(j\).

3. **Vincoli di Ottimizzazione:**
   - Somma dei pesi: $\sum_{i=1}^n w_i = 1$ (vincolo di completezza).
   - Pesi non negativi ($w_i \geq 0$) se non è consentita la vendita allo scoperto.

4. **Risultato:**  
   Risolvendo il problema di ottimizzazione, otteniamo la frontiera efficiente come una curva concava che mostra la relazione tra rischio (asse \(x\)) e rendimento atteso (asse \(y\)).

---

### **Sharpe Ratio e Allocazione Ottimale**
Lo **Sharpe Ratio** è una misura chiave per valutare la performance di un portafoglio. Esso quantifica quanto rendimento extra un investitore ottiene per ogni unità di rischio assunto, rispetto a un asset privo di rischio. La formula è:
\[
$$\text{Sharpe Ratio} = \frac{E(R_p) - R_f}{\sigma_p}$$
\]
Dove:
- $E(R_p)$: Rendimento atteso del portafoglio.
- $R_f$: Tasso privo di rischio (es. rendimento dei titoli di stato).
- $\sigma_p$: Volatilità del portafoglio.

Un portafoglio con un elevato Sharpe Ratio è preferibile perché offre un buon equilibrio tra rischio e rendimento.

---

### **Portafoglio di Mercato e Linea del Mercato dei Capitale (CML)**
1. **Portafoglio di Mercato:**  
   Quando si introduce la possibilità di investire in un asset privo di rischio ($R_f$), il portafoglio ottimale sulla frontiera efficiente è quello che massimizza lo Sharpe Ratio. Questo portafoglio è noto come **portafoglio di mercato**.

2. **Linea del Mercato dei Capitali (CML):**  
   La CML è una linea retta che parte dal tasso privo di rischio ($R_f$) e si estende tangenzialmente alla frontiera efficiente. La pendenza della CML è lo Sharpe Ratio del portafoglio di mercato.

---

### **Significato Pratico**
1. **Perché è importante la frontiera efficiente?**
   - Aiuta gli investitori a identificare il mix ottimale di asset in base alle proprie preferenze di rischio-rendimento.
   - Fornisce una base scientifica per la diversificazione.

2. **Limitazioni:**
   - Dipende da stime future di rendimenti, rischi e covarianze, che possono essere imprecise.
   - Non tiene conto di eventi estremi o rischi sistemici.

3. **Utilizzo nella costruzione del portafoglio:**
   - Gli investitori possono scegliere portafogli lungo la frontiera efficiente in base alla loro propensione al rischio.
   - In combinazione con asset privi di rischio, gli investitori possono creare portafogli lungo la CML.

---

### **Conclusione**
La frontiera efficiente fornisce un quadro teorico per ottimizzare portafogli e prendere decisioni di investimento basate sul trade-off rischio-rendimento. Sebbene presenti alcune limitazioni, rimane un concetto fondamentale per comprendere la diversificazione e la gestione del rischio nei mercati finanziari.

In [ ]:
from trading_system_py.analysis.asset_allocation.efficient_frontier import EfficientFrontier
ef = EfficientFrontier(data=data_retriever.data)
ef.plot_generated_portfolios()

In [ ]:
ef.generated_portfolios.head()

In [ ]:
ef.plot_efficient_frontier()

In [ ]:
ef.get_individual_stock_return_volatility()

In [ ]:
ef.get_max_sharpe_random_portfolios()

In [ ]:
ef.get_min_volatility_random_portfolios()

In [ ]:
ef.get_optimized_max_sharpe()

In [ ]:
ef.get_optimized_min_volatility()